In [10]:
# https://towardsdatascience.com/python-how-to-get-live-market-data-less-than-0-1-second-lag-c85ee280ed93

In [3]:
import numpy as np
import pandas as pd

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

In [8]:
# Interval required 5 minutes
data = yf.download(tickers='AAPL', period='5d', interval='5m')
#Print data
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-03-15 09:30:00-04:00,121.339996,121.349998,120.419998,120.889999,120.889999,4450822
2021-03-15 09:35:00-04:00,120.879997,121.330002,120.800003,120.955002,120.955002,1684290
2021-03-15 09:40:00-04:00,120.949997,121.680000,120.750000,121.560799,121.560799,2152141
2021-03-15 09:45:00-04:00,121.570000,122.290001,121.529999,122.175003,122.175003,2264924
2021-03-15 09:50:00-04:00,122.169998,123.489998,122.144997,123.169701,123.169701,3420954
...,...,...,...,...,...,...
2021-03-19 15:35:00-04:00,120.154999,120.296898,120.110001,120.217003,120.217003,1685847
2021-03-19 15:40:00-04:00,120.209999,120.339996,120.180000,120.330101,120.330101,1512792
2021-03-19 15:45:00-04:00,120.334999,120.410004,120.149902,120.280098,120.280098,1735608


In [9]:
type(data)

pandas.core.frame.DataFrame